In [8]:
import os
import pandas as pd

import pickle
import re

from collections import Counter

In [2]:
data_path = 'C://Users/LGCNS/Documents/GitHub/Q_Bert/dt/wiki_kor_210520_pages-articles/'

# file_name = os.listdir(data_path)[-1]

In [5]:
# file_path = os.path.join(data_path, file_name)

# tree = parse(file_path)
# root = tree.getroot()

# 데이터 크기가 매우 크고, Local Disk 환경이 너무 작아 데이터를 업로드하는데 문제가 많습니다.
# 이에 따라 Opensource를 활용하여 Wiki Unzip(Extract)하겠습니다.

# https://github.com/attardi/wikiextractor

# !pip install wikiextractor

# python -m wikiextractor.WikiExtractor kowiki-20210520-pages-articles.xml

# Windows에서 WikiExtractor는 버그가 있어 사용하지 못합니다. Linux 응용프로그램을 설치하여 데이터를 준비했습니다.
# Warning: problems have been reported on Windows due to poor support for StringIO in the Python implementation on Windows.

In [3]:
# Recursively Get File Path
# 파일 경로만 확인하여 가져옵니다.

def get_recursive_files(file_path) :
    
    result_lst = []
    
    file_list = os.listdir(file_path)
    
    for f in file_list :
        
        target = os.path.join(file_path, f)
        
        if os.path.isdir(target) :
            
            result_lst += get_recursive_files(target)
        
        else :
            
            result_lst.append(target)
    
    return result_lst
    

In [4]:
full_path_lst = get_recursive_files(os.path.join(data_path, 'text'))

In [5]:
print(len(full_path_lst)) # 총 738개 파일

793


In [7]:
# 데이터 한 개를 불러와 데이터를 확인합니다.

test_file = full_path_lst[1]

with open(test_file, 'r', encoding = 'utf8') as f:
    txts = f.read()

In [8]:
import re

def clean_html(raw_html):
    
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def line_split(line) :
    
    all_lines = line.split('\n')
    
    filtered = list(filter(lambda x: x.strip()!='', all_lines))
    
    return filtered

In [10]:
def prep_txt(file_path) :
    
#     print(file_path, '\tSTART')
    
    global txt_dict
    
    #0. 불러오기
    with open(file_path, 'r', encoding = 'utf8') as f:
        txts = f.read()
    
    #1. <tag 제거>
    txt_offtag = clean_html(txts)

    #2. <\n\n\n\n> 토큰 단위 split
    txt_offtag_split_documents = txt_offtag.split('\n\n\n\n')

    #3. <\n> Filtering, 그리고 Dict화
    txt_split = list(map(line_split, txt_offtag_split_documents))

    for txt_s in txt_split :
        
        if len(txt_s) > 1 :
            txt_dict[txt_s[0]]= txt_s[1:]
#     print(file_path, "\tEND")
    return True

In [11]:
# 파일크기가 크므로 반으로 나누어 저장하도록 한다.

half_len = len(full_path_lst) // 2

In [21]:
txt_dict = {}

# 각각의 파일은 여러개의 Page가 포함된 Documents이다.
# 줄바꿈표시, <doc> </doc>과 같은 태깅이 포함되어있다.

# \n\n\n\n Token 별로 Documents를 형성하는 것으로 보임
# 소제목과 중제목 등이 포함되어있음. Min_Seq_Len를 이용한 Filtering

for path_one in full_path_lst[:half_len] :
    prep_txt(path_one)

with open(os.path.join(data_path, 'wiki_ko_dict1.pkl'), 'wb') as f:
    pickle.dump(txt_dict, f)

In [22]:
txt_dict = {}

for path_one in full_path_lst[half_len:] :
    prep_txt(path_one)

with open(os.path.join(data_path, 'wiki_ko_dict2.pkl'), 'wb') as f:
    pickle.dump(txt_dict, f)

## Sentence Split

In [96]:
with open(os.path.join(data_path, 'wiki_ko_dict2.pkl'), 'rb') as f:
    txt_dict = pickle.load(f)

In [40]:
len(list(txt_dict.keys()))

356970

In [38]:
list(txt_dict.keys())[0]

'디어 보이즈'

In [83]:
list(txt_dict.keys())[190:200]

['Pcap',
 '이야기쇼 두드림',
 '코담배',
 '비연호',
 '수집',
 '골동품',
 'S-VHS',
 '너트크래커',
 '코어 오디오',
 '피규어']

In [77]:
# next Sentence 분석을 위해 문장 단위로 Split하여야한다.
# 작성자에 따라 문장이 Split되어 있거나 여러 문장이 한 문장으로 표현되어 있거나 한다.
# 소제목 등이 존재한다.
# 종종 utf8 기준 encoding이 깨져있는 경우도 존재한다.
# 

# '.'로 Split하면 대체로 잘 되나 소수점에 의하여 Sentence가 나뉘기도 한다.                # 솔나 시
# Sentence가 한 개 밖에 경우도 존재한다.                                                  # ISDA

# 괄호 안을 남겨둘 필요가 있을까?

# ''.join(txt_dict['구아마텔라']).split('.')

In [99]:
re.sub(r'\w[.]\w', '__', txt_dict['유승필'][1])

'1963년 서울고 2년 수학 후 미국 유학, 1965년 Diploma Woldorf School 졸업,1965년 - 1969년에는 __. Heidelberg College 수학, 경제학 전공.'

In [6]:

# 0. ';'가 포함된 경우 wiki tag가 남아있는 경우로 데이터 제거
#    'colspan'의 경우 테이블 데이터가 남아있는 경우로 데이터 제거
# 1. 영어 대문자는 모두 소문자로 치환 (Uncased)
# 2. 괄호 및 괄호 안에 있는 내용 제거  <---- 21. 05. 31. PATCH 괄호 안도 의미를 가질 수 있다고 생각하여 남김
# 3. N.S.Vierson 등 예외처리
# 4. 소수점 토큰 예외처리
# 5. 1. ~~~, 2. ~~~ 예외처리
# 6. \xa0 (character decoding 문제), 공백으로 변환
# 7. 연속된 공백 처리
# 8. sp. --> speices의미로 예외처리
# 9. '.' 으로 split
# 10. 문장 strip
# 11. 공백문장 제거
# 12. 문장은 최소 min_seq_len(3) 길이 이상이 되어야 함.
# 13. 예외처리 (__)를 다시 (.)으로 변환

def transfer_sentences(lst, min_seq_len = 2) :
    
    def cleaning_except(raw_txt):
        
        def dot_to_underbar(m) :
            tmp_str = m.group()
            return tmp_str.replace('.', '__')
        
#         cleantext = re.sub(r'\([^)]*\)', '', raw_txt) 
        cleantext = re.sub(r'[a-z]+[.]+[a-z]+[.]*[a-z]*[.]*', dot_to_underbar, raw_txt)
        cleantext = re.sub(r'[1-9]+[.]+', dot_to_underbar, cleantext)
        cleantext = re.sub(r'\d[.]\d', dot_to_underbar, cleantext)
        
        return cleantext
    
    
    
    total_txt = ''.join(lst)
    
    if (';' in total_txt) | ('colspan' in total_txt) :
        return []
    
    total_txt = total_txt.lower()
    total_txt = cleaning_except(total_txt)
    total_txt = total_txt.replace('\xa0', ' ')
    total_txt = total_txt.replace('  ', ' ')
    total_txt = total_txt.replace('sp.', 'sp__')
    
    total_txts = total_txt.split('.')
    
    total_txts = list(map(lambda x : x.strip(), total_txts))
    total_txts = list(filter(lambda x : x != '', total_txts))
    total_txts = list(filter(lambda x : len(x.split(' ')) >= min_seq_len, total_txts))
    total_txts = list(map(lambda x : x.replace('__', '.'), total_txts))
    
    return total_txts

In [9]:
with open(os.path.join(data_path, 'wiki_ko_dict1.pkl'), 'rb') as f:
    txt_dict_pre = pickle.load(f)

print(len(list(txt_dict_pre.keys())))
    
txt_dict_post = {}    
for key in txt_dict_pre.keys() :
    tmp_txt = transfer_sentences(txt_dict_pre[key], 3)
    tmp_txt = list(map(lambda x : x+".", tmp_txt))
    # 문장이 한 개 이상인 Documents만 사용
    if len(tmp_txt) > 1 :
        txt_dict_post[key] = tmp_txt

print(len(list(txt_dict_post.keys())))
        
with open(os.path.join(data_path, 'wiki_ko_dict1_p.pkl'), 'wb') as f:
    pickle.dump(txt_dict_post, f)

176610
135099


In [13]:
with open(os.path.join(data_path, 'wiki_ko_dict2.pkl'), 'rb') as f:
    txt_dict_pre = pickle.load(f)

print(len(list(txt_dict_pre.keys())))
    
txt_dict_post = {}    
for key in txt_dict_pre.keys() :
    tmp_txt = transfer_sentences(txt_dict_pre[key], 3)
    tmp_txt = list(map(lambda x : x+".", tmp_txt))
    # 문장이 한 개 이상인 문장만 사용
    if len(tmp_txt) > 1 :
        txt_dict_post[key] = tmp_txt

print(len(list(txt_dict_post.keys())))
        
with open(os.path.join(data_path, 'wiki_ko_dict2_p.pkl'), 'wb') as f:
    pickle.dump(txt_dict_post, f)

356970
224398


## Sample 100

In [10]:
with open(os.path.join(data_path, 'wiki_ko_dict1_p.pkl'), 'rb') as f:
    txt_dict_post = pickle.load(f)

In [11]:
import numpy as np


key_lst = list(txt_dict_post.keys())
total_len = len(key_lst)



r_indexs = np.random.randint(0, total_len, 100)

In [12]:
for r_index in r_indexs :
    
    key = key_lst[r_index]
    
    print("===================================================")
    print(key)
    print(txt_dict_post[key])

관산진
['관산 진(, )은 중화민국 타이둥 현의 진이다.', '넓이는 58.7351 km2이고, 인구는 2015년 8월 기준으로 8,974명이다.', '관산 진은 타이둥 현 북부에 위치해, 서쪽은 하이돤 향과 북쪽은 츠사 향과 동쪽은 둥허 향과 남쪽은 루예 향과 각각 접하고 있다.', '동쪽의 해안 산맥, 서쪽의 중양 산맥에 끼어 있는 화둥 종곡 평원 남부에 위치하고 진내를 베이난시(卑南渓)가 흐르고 있다.', '연 강수량은 약 2,000mm, 연평균 기온은 23.7℃이다.']
티어즈 투 티아라 -화관의 대지-
['티어즈 투 티아라 -화관의 대지-()는 2008년 7월 17일에 플레이스테이션 3로 발매된 아쿠아플러스 제작의 게임이다.', '리프의 tears to tiara의 가정용 게임기 이식 작품이지만 캐릭터 디자인과 게임 시스템을 비롯한 상당수의 요소를 교체하는 등 전면적인 풀리메이크를 단행하여 사실 상 새로운 게임이라고 해도 좋을 정도의 변화를 보이고 있다.', '아울러 플레이스테이션 3로 발매되는 첫 번째 pc판 원작 미소녀 게임의 가정용 게임기 이식작이면서 첫 번째 본격적인 ps3용 미소녀 게임이라는 기록도 가지고 있다.', '참고로 본 게임을 바탕으로 2009년 4월 tv 애니메이션화도 이뤄졌는데 이에 대해션 원작내의 항목을 참조.', '게임 진행 형식의 경우 오리지널 pc판과 마찬가지로 어드벤처 게임 부분과 시뮬레이션rpg 부분을 교대로 바꾸어가면서 진행하는 형식의 게임이다.', '단, 시뮬레이션 부분의 경우 오리지널 pc판이 하프 리얼타임 방식의 전투인 데 반해 ps3판의 경우『칭송받는 자』와 마찬가지의 정통 srpg 스타일의 전투로 바뀌었다.', '2008년 10월 3일자로 1.01 업데이트가 공개 되었다.', '본 업데이트는 플레이스테이션 3가 네트워크에 연결이 되어있을 경우 게임 기동시 자동으로 업데이트가 진행되며, 업데이트를 통해 변경·개선된 점은 다음과 같다.', '일본 psn을 통해 시스템 테마가 판매 중에 있다.']
대한민국 철도청

## Sentence로 저장

In [2]:
import tqdm
tqdm.tqdm_notebook

In [8]:
import os
import pickle
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

data_path = 'C://Users/LGCNS/Documents/GitHub/Q_Bert/dt/wiki_kor_210520_pages-articles/'

In [4]:
with open(os.path.join(data_path, 'wiki_ko_dict1_p.pkl'), 'rb') as f:
    dict1 = pickle.load(f)

with open(os.path.join(data_path, 'wiki_ko_dict2_p.pkl'), 'rb') as f:
    dict2 = pickle.load(f)

In [5]:
# # memory 문제가 있음.
# sentence1 = sum(list(dict1.values()), [])

# sentence = sum(list(dict1.values())[0:10000], [])

In [9]:
total_sentence_num = 0
unique_word = set()

for sentences in tqdm_notebook(list(dict1.values())) :
    
    for sentence in sentences :
        
        total_sentence_num+= 1
#         unique_word = set(list(unique_word) + sentence.split(' '))
        
        with open('./dt/train_wiki_sentence1.txt', 'a', encoding = 'utf8') as f:

            f.write(sentence+'\n')


  0%|          | 0/135099 [00:00<?, ?it/s]

In [10]:
print(total_sentence_num)
# print(len(unique_word))

2037700


In [ ]:
for sentences in tqdm_notebook(list(dict2.values())) :
    
    for sentence in sentences :
        
        total_sentence_num+= 1
#         unique_word = set(list(unique_word) + sentence.split(' '))
        
        with open('./dt/train_wiki_sentence2.txt', 'a', encoding = 'utf8') as f:

            f.write(sentence+'\n')


  0%|          | 0/224398 [00:00<?, ?it/s]

In [ ]:
print(total_sentence_num)
# print(len(unique_word))